In [1]:
import os
import openai
import subprocess
import requests
from typing import List, Optional
from pprint import pprint

API_KEY = os.getenv("OPENAI_API_KEY")
GPT_MODEL = ["text-davinci-003"]

def print_response(response):
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        error = response.json()
        print(f"Error: {error['error']}")
        return error


## Create completion
- DOC url: https://platform.openai.com/docs/api-reference/completions/create

# Making requests

In [2]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}",
}

response = requests.post("https://api.openai.com/v1/completions", \
    headers=headers, json={
        "model": "text-davinci-003",
        "prompt": "Say this is a test",
        "temperature": 0.2,
        "max_tokens": 1000
    }
)

def create_completion(prompt:str, suffix:str = None, model: str = "text-davinci-003", max_tokens: Optional[int] = 500, temperature: float = 1, top_p: Optional[int] = 1, n: Optional[int] = 1,  stream: Optional[bool] = False, echo: Optional[bool] = False):
    """
    https://platform.openai.com/docs/api-reference/completions/create
    ##### Given a prompt, the model will return one or more predicted completions, and can also return the probabilities of alternative tokens at each position.

    Args:
        - prompt (str): 
            - The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays. 
            Note that <|endoftext|> is the document separator that the model sees during training, so if a prompt is not specified the model will generate as if from the beginning of a new document. 
        
        - suffix (str): 
            - The suffix that comes after a completion of inserted text.
        
        - model (str, required): 
            - ID of the model to use. You can use the **List models** API to see all of your available models, or see our **Model overview** for descriptions of them. Defaults to "text-davinci-003".
        
        - max_tokens (int, optional, Default: 500): 
            - The maximum number of tokens to generate in the completion.
                The token count of your prompt plus `max_tokens` cannot exceed the model's context length. Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Defaults to 500.
        
        - temperature (float, optional): 
            - What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
            We generally recommend altering this or `top_p` but not both. Defaults to 0.
            
        - top_p (float, optional):
            - An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
            - We generally recommend altering this or `temperature` but not both. Defaults to 1.
            
        - n (int, optional, Default: 1):
            - The number of completions to generate. Defaults to 1. 
        
        - stream (bool, optional): 
            - Whether to stream back partial progress. If set, tokens will be sent as data-only server-sent events as they become available, with the stream terminated by a `data: [DONE]` message. Defaults to False.
        
        - echo (bool, optional): _description_. Defaults to False.
        

    Returns:
        _type_: _description_
    """    
    response = requests.post("https://api.openai.com/v1/completions", \
        headers=headers, json={
            "model": model,
            "prompt": prompt,
            "suffix": suffix,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stream": stream,
            "echo": echo,
            "stop": suffix,
            "n": n,
            "top_p": top_p
        }
    )
    return response
    
def edit_prompt(instruction: str, input: Optional[str] = None, model: str = "text-davinci-edit-001", num: Optional[int] = 1, temperature: Optional[float] = 1, top_p: Optional[float] = 1):
    """#### Given a prompt and an instruction, the model will return an edited version of the prompt.
    https://platform.openai.com/docs/api-reference/edits
    Args:
        - instruction (str, required): 
            - The instruction that tells the model how to edit the prompt.
            
        - input (Optional[str], optional): 
            - The input text to use as a starting point for the edit. Defaults to None.
        
        - model (str, required): 
            - ID of the model to use. You can use the `text-davinci-edit-001` or `code-davinci-edit-001` model with this endpoint. Defaults to "text-davinci-edit-001".

        - n (Optional[int], optional): 
            - How many edits to generate for the input and instruction. Defaults to 1.

        - temperature (Optional[float], optional): 
            - What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
            - We generally recommend altering this or `top_p` but not both. Defaults to 1.
        
        - top_p (Optional[float], optional):
            - An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
            - We generally recommend altering this or `temperature` but not both. Defaults to 1.

    Returns:
        - response
    """    
    response = requests.post("https://api.openai.com/v1/edits", \
        headers=headers, json={
            "model": model,
            "input": input,
            "instruction": instruction,
            "n": num,
            "temperature": temperature,
            "top_p": top_p
        }
    )
    return response

## load kr_common

In [5]:
import json

with open("./test/common_kr.json", "r") as f:
    kr_common = json.load(f)
pprint(kr_common.keys())
print()

dict_keys(['landing', 'commonErrors', 'buttons', 'navbar', 'generate-page', 'upscaler', 'purchase', 'lg-profile', 'lg-payments', 'settings', 'feedback', 'sm-profile', 'sm-payments', 'sm-settings', 'sm-security', 'footer', 'login', 'signup', 'successfull', 'verification'])



In [33]:
completion_text = """
The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.

Human: Hello, who are you?
AI:
"""

res_comp = create_completion(completion_text, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)

In [34]:
pprint(res_comp.json(), indent=2)

{ 'choices': [ { 'finish_reason': 'stop',
                 'index': 0,
                 'logprobs': None,
                 'text': "Hello there! I'm an AI assistant. I'm here to help "
                         'you with any questions or tasks you may have.'}],
  'created': 1675942208,
  'id': 'cmpl-6hzPs4EX8OP1LI1wURc6t4vVkyPKZ',
  'model': 'text-davinci-003',
  'object': 'text_completion',
  'usage': {'completion_tokens': 24, 'prompt_tokens': 38, 'total_tokens': 62}}


In [36]:
completion_text_conversation = """
Convert movie titles into emoji.

Back to the Future: 👨👴🚗🕒 
Batman: 🤵🦇 
Transformers: 🚗🤖 
Star Wars:
"""
res_comp = create_completion(completion_text_conversation, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
pprint(res_comp.json()['choices'][0]['text'], indent=2)

'✨🌌🤴👸'


In [38]:
completion_text_summarization = """
Summarize this for a second-grade student:

Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows,[20] and is on average the third-brightest natural object in the night sky after the Moon and Venus.
"""
res_comp = create_completion(completion_text_summarization, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
pprint(res_comp.json()['choices'][0]['text'], indent=2)

('\n'
 'Jupiter is the fifth planet from the Sun and the biggest in our Solar '
 'System. It is very bright and can be seen in the night sky. It has been '
 'known since ancient times and is named after the Roman god Jupiter. It is so '
 'bright that it can cast shadows on Earth.')


In [39]:
completion_text_episode = """
Write an episode for 2-minute speaking practice English under 100 words. This episode is for blog posts in markdown format. heading has 3 hashes. You are given main subject and subtopic as [main subject, subtopic]

[Amusement Park, Parade is more fun than rides]: 
"""
res_comp = create_completion(completion_text_episode, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
pprint(res_comp.json(), indent=2)

{ 'choices': [ { 'finish_reason': 'stop',
                 'index': 0,
                 'logprobs': None,
                 'text': '\n'
                         '### Parade is more Fun than Rides\n'
                         '\n'
                         'Going to an amusement park is always an exciting '
                         'experience. But for me, the best part is the parade. '
                         'The colorful floats, the music, and the performers '
                         'make it a truly magical experience. The rides are '
                         "fun, but the parade is much more entertaining. It's "
                         'a great way to get the whole family involved in the '
                         "fun. Plus, it's a great way to get a break from the "
                         "heat and the crowds. So, if you're looking for a fun "
                         "day out, don't forget to check out the parade!"}],
  'created': 1675942794,
  'id': 'cmpl-6hzZK1YgBVGKvqoxzKd

In [40]:
completion_text = """
I just created a new Github repository. How can I connect this repository to my local git?

Description: 
"""
res_comp = create_completion(completion_text, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
pprint(res_comp.json(), indent=2)

{ 'choices': [ { 'finish_reason': 'stop',
                 'index': 0,
                 'logprobs': None,
                 'text': '\n'
                         '1. Open your terminal and navigate to the local '
                         'directory where you want to store the repository.\n'
                         '\n'
                         '2. Run the command `git init` to initialize a new '
                         'local git repository.\n'
                         '\n'
                         '3. Run the command `git remote add origin '
                         '<url-of-your-github-repository>` to connect your '
                         'local repository to the remote Github repository.\n'
                         '\n'
                         '4. Run the command `git pull origin master` to pull '
                         'the contents of the remote repository into your '
                         'local repository.\n'
                         '\n'
                         '5. Ru

# Edit prompt

In [18]:
example_code = """
How come I use my phone to check the weather? My teacher said that I should use.
"""

In [19]:
edit_res = edit_prompt("Fix the sentence if the it looks awkward.", input=example_code, model="text-davinci-edit-001", num=1, temperature=0.2, top_p=1)
print(edit_res.json())
print(''.join(edit_res.json()['choices'][0]['text']))

{'object': 'edit', 'created': 1675941262, 'choices': [{'text': '\nHow come I use my phone to check the weather? My teacher said that I should use the computer.\n', 'index': 0}], 'usage': {'prompt_tokens': 41, 'completion_tokens': 52, 'total_tokens': 93}}

How come I use my phone to check the weather? My teacher said that I should use the computer.

